In [2]:
"""
Use Bokeh to visualize mod predictions

input files:
- tRNA mod calls tab-delimited text file
- tRNA mod scores text file
- protein prediction file

outputs:
- interactive html file bilt using bokeh

"""

def parseInput():
    """Parse command line input and output files"""
    import argparse
    
    #Argparse setup
    argParser = argparse.ArgumentParser(description = 'This program generates track hubs that display Modomics data')
    argParser.add_argument('-c', '--mod_calls', required = True, help = 'modification calls file')
    argParser.add_argument('-o', '--output_file', required = True, help = 'output html file')
    argParser.add_argument('-k', '--kingdom', required = False, choices = {'E', 'A', 'B'}, help = 'Domain of organism')
    argParser.add_argument('-s', '--mod_scores', required = True, help = 'modification scores file')
    argParser.add_argument('-e', '--enzyme_predictions', required = True, help = 'enzyme predictions file')
    
    kingdoms = {'E': 'Eukaryota', 'A': 'Archaea', 'B': 'Bacteria'}
    
    clArgs = argParser.parse_args('-c ./strepneumo_predictions/modCalls.txt -s ./strepneumo_predictions/modScores.txt -e ./strepneumo_predictions/output.tsv -o ./strepneumo_predictions'.split())
    callFile = clArgs.mod_calls
    #orgKing = kingdoms[clArgs.kingdom]
    outFile = clArgs.output_file
    scoreFile = clArgs.mod_scores
    protFile = clArgs.enzyme_predictions
    
    return callFile, scoreFile, protFile, outFile

def readTSV(inputFile):
    """Read a TSV with row and column headers"""
    
    #Store mod info: mod: {modified_position: {'+': [tRNAs], '-': [tRNAs]]}
    seqsDict = {}
    
    with open(inputFile, 'r') as inF:
        
        lines = inF.readlines()
        
        #TSV columns
        cols = lines[0].strip().split('\t')
        
        for line in lines[1:]:
            splitLine = line.strip().split('\t')
            
            #tRNA name
            tRNAname = splitLine[0]
            
            #Make dictionary of sprinzl positions
            seqsDict[tRNAname] = {}
            
            #Iterate through sprinzl positions
            for sprinzlPos, base in zip(cols[1:], splitLine[1:]):
                
                seqsDict[tRNAname][sprinzlPos] = base
                    
        inF.close()
    
    return seqsDict

def bit2prob(bitScore):
    """Convert bit score to probability, assuming a binary outcome"""
    
    return 1/(1+2**(-bitScore))
    


def makeFig(callDict, scoreDict, protDict, outF):
    """Make an interactive cloverleaf output figure"""
    from bokeh.plotting import figure, show, save
    from bokeh.models import ColumnDataSource, Grid, LinearAxis, Plot, Text, ColorBar

    """
    Goals for this code
    - Get rid of axes and grid lines
    - adjust sizes of circles for variable loop
    - add key for heat map
    - general fine-tuning
    """
    
    
    from tRNAinfo import cloverCoords
    
    #Iterate through tRNA 120.418
    for tRNA in callDict.keys():
        
        #Generate data vectors to use in the plot
        #bases = list([callDict[tRNA][pos] for pos in cloverCoords.keys()])
        #scores = list([scoreDict[tRNA][pos] for pos in cloverCoords.keys()])

        #Store data vetors
        X = []
        Y = []
        col = []
        scores = []
        bases = []
        probs = []
        positions = []
        sizes = []
        
        #Add different values to different vectors
        for pos in cloverCoords.keys():
            
            positions.append(pos)
            bases.append(callDict[tRNA][pos])
            
            X.append(cloverCoords[pos][0])
            Y.append(-cloverCoords[pos][1])
            
            #Select color
            if scoreDict[tRNA][pos] != '-':
                
                #append probabilities
                prob = bit2prob(float(scoreDict[tRNA][pos]))
                probs.append(prob)
                scores.append(scoreDict[tRNA][pos])
                
                #Color the heat map based on mod score
                if prob > 0.5:
                    col.append((255, 255-(200*(prob-0.5)), 255-(200*(prob-0.5))))
                
                elif prob < 0.5:
                    col.append((255-(200*(0.5-prob)), 255-(200*(0.5-prob)), 255))
                
                else:
                    col.append((200, 200, 200))
                    
            else:
                probs.append('None')
                scores.append('None')
                col.append((255, 255, 255))
                
            #Add size of circle
            if pos[0] == 'e':
                sizes.append(cloverCoords[pos][2]+12)
            else:
                sizes.append(cloverCoords[pos][2]+13)
        
        cloverSource = ColumnDataSource({'x': X, 'y': Y, 'col': col, 
                                         'bases': bases, 'scores': scores, 
                                         'probs': probs, 'positions': positions, 
                                         'sizes': sizes})
        
        Tooltips = [('modification', '@bases'), 
                    ('score', '@scores'),
                    ('position', '@positions')]
                    #('possible enzymes', 'placeholder')]
        
        #Instantiate cloverleaf structure 
        clover = figure(title = '{0}'.format(tRNA), x_axis_label = '', y_axis_label = '',
                        toolbar_location=None, tools = 'hover', tooltips = Tooltips)
        
        #Plot circles
        sprinzlCircle = clover.circle('x', 'y', size = 'sizes', fill_color = 'col',
                                      line_color = 'grey', name = 'positions', source = cloverSource)
            
        #Plot text
        baseText = Text(x = 'x', y = 'y', 
                        text = 'bases', text_color = 'black', 
                        text_align = 'center', text_baseline = 'middle')
        clover.add_glyph(cloverSource, baseText)
        
        
        #Remove excess visual info from default plotting
        clover.xaxis.major_tick_line_color = None
        clover.yaxis.major_tick_line_color = None
        clover.xaxis.minor_tick_line_color = None
        clover.yaxis.minor_tick_line_color = None
        clover.xgrid.grid_line_color = None
        clover.ygrid.grid_line_color = None
        clover.xaxis.major_label_text_font_size = '0pt'
        clover.yaxis.major_label_text_font_size = '0pt'
            
        save(clover, filename = '{0}/{1}.html'.format(outF, tRNA))
        
        

    
    
    

def main():
    import pandas as pd
    from tRNAinfo import sprinzl2coords
    
    
    #Recieve input commands
    calls, scores, proteins, outFile = parseInput()
    
    callsDict = readTSV(calls)
    scoresDict = readTSV(scores)
    
    makeFig(callsDict, scoresDict, proteins, outFile)
    
    
    
    
main()
    